# Deep reinforcement learning for the optimization of traffic light control with real-time data


## Instructions for running

Below you can find the description of the parameters you can use for the simulation class or wrapper for training a DDQN in a preset SUMO environment.

Also, you can find a couple of self-explanatory examples of how two run a single scenario training or gridsearch.

### Parameters

#### DDQN parameters

**q_network** : (str) keras model instance to predict q-values for current state ('simple' or 'linear')

**gamma** : (int) discount factor for rewards

**target_update_freq** : (int) defines after how many steps the q-network should be re-trained

**train_freq**: (int) How often you actually update your Q-Network. Sometimes stability is improved
    if you collect a couple samples for your replay memory, for every Q-network update that you run.

**num_burn_in** : (int) defines the size of the replay memory to be filled before, using a specified policy

**batch_size** : (int) size of batches to be used to train models

**optimizer** : (str) keras optimizer identifier ('adam')

**max_ep_len** : (int) stops simulation after specified number of episodes

**experiment_id** : (str) ID of simulation

**model_checkpoint** : (bool) store keras model checkpoints during training

**policy** : (str) policy to choose actions ('epsGredy', 'linDecEpsGreedy', 'greedy' 'randUni')

**eps** : (float) exploration factor
    if policy = 'linDecEpsGreedy' -> The epsilon will decay from 1 to eps
    if policy = 'epsGredy' -> eps to evaluate eps policy
    


#### Environment parameters

**network** : (str) network complexity ('simple' or 'complex')

**demand**: (str) demand scenario ('rush' or 'nominal')

**use_gui** : (bool) wether to use user interface

**delta_time** : (int) simulation time between actions

**reward** : type of reward. ('balanced' or 'negative')

#### Memory buffer parameters

 
**max_size** : (int) memory capacity required

#### Additional parameters


**num_episodes** : (int) number of episodes to train the algorithm. THis can also be changed in train method.

**eval_fixed** = (bool) Evaluate fixed policy during training. Used for plotting

**monitoring** : (bool) store episode logs in tensorboard

**episode_recording** : (bool) store intra episode logs in tensorboard

**seed** = (int)

In [30]:
# IMPORTS
##########################

%load_ext autoreload
%autoreload 2


import simulation
import plotting
import tools
import glob

import pandas as pd
import os
import json

# import tensorflow as tf
# tf.logging.set_verbosity(tf.logging.ERROR)

import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (12,12)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Example 1:  Single run

#### Instantiate

In [ ]:
param = { 
    "experiment_id" : "Test",
    
    "q_network_type" : 'simple', #"linear"
    "gamma" : 0.99,
    "target_update_freq" : 5000,
    "train_freq" : 1,
    "num_burn_in" : 200,
    "batch_size" : 30,
    "optimizer" : 'adam', 
    
    "max_ep_length" : 1000,
    "policy" : "linDecEpsGreedy", #"epsGreedy"    
    "eps" : 0.1,
    
    "network" : "complex", #simple
    "demand" : "rush", #nominal
    
    "use_gui" : False,
    "delta_time" : 10,
    
    "reward" : "balanced", #negative
    
    "max_size" : 100000,
    "num_episodes" : 2,
    
    "eval_fixed" : False,
    "episode_recording" : False,
    "model_checkpoint" : True,
 
}


sumo_RL = simulation.simulator(**param)
log_path = "./logs/"+ param["experiment_id"]

#### Train

In [ ]:
res = sumo_RL.train()
pd.DataFrame(res)

#### Evaluate

In [ ]:
_ = sumo_RL.evaluate(runs=3, use_gui=False)

#### Plot results

In [ ]:
plotting.plot_evaluation(log_path)

### Experiment 2: Hyperparameter grid search

#### Instantiate

In [6]:
param = {
    
    "experiment_id" : ["Test_gridsearch"],
    "batch_size" : [30,50],
    "target_update_freq" : [5000,10000],
    "gamma" : [0.99],
    "train_freq" : [1],
    "max_size" : [10000],
    "max_ep_length" : [1000],
    "policy" : ["epsGreedy","linDecEpsGreedy"],
    "eps" : [0.1],
    "delta_time" : [10],
    "reward" : ["balanced"],
    "network" : ["simple"],
    "num_episodes" : [30],
}

log_path = "./logs/"+ param["experiment_id"][0]

#### Run gridsearch

In [18]:
param_grid = tools.iter_params(**param)
tools.gridsearch(list(param_grid), log_path)

[(0, {'experiment_id': 'Test_gridsearch', 'batch_size': 30, 'target_update_freq': 5000, 'gamma': 0.99, 'train_freq': 1, 'max_size': 10000, 'max_ep_length': 1000, 'policy': 'epsGreedy', 'eps': 0.1, 'delta_time': 10, 'reward': 'balanced', 'network': 'simple', 'num_episodes': 30}), (1, {'experiment_id': 'Test_gridsearch', 'batch_size': 30, 'target_update_freq': 5000, 'gamma': 0.99, 'train_freq': 1, 'max_size': 10000, 'max_ep_length': 1000, 'policy': 'linDecEpsGreedy', 'eps': 0.1, 'delta_time': 10, 'reward': 'balanced', 'network': 'simple', 'num_episodes': 30}), (2, {'experiment_id': 'Test_gridsearch', 'batch_size': 30, 'target_update_freq': 10000, 'gamma': 0.99, 'train_freq': 1, 'max_size': 10000, 'max_ep_length': 1000, 'policy': 'epsGreedy', 'eps': 0.1, 'delta_time': 10, 'reward': 'balanced', 'network': 'simple', 'num_episodes': 30}), (3, {'experiment_id': 'Test_gridsearch', 'batch_size': 30, 'target_update_freq': 10000, 'gamma': 0.99, 'train_freq': 1, 'max_size': 10000, 'max_ep_length':

Run 8 -- running episode 3 / 30
Run 1 -- running episode 4 / 30
Run 3 -- running episode 4 / 30
Run 2 -- running episode 4 / 30
Run 1 -- running episode 5 / 30
Run 4 -- running episode 4 / 30
Run 5 -- running episode 4 / 30
Run 7 -- running episode 4 / 30
Run 8 -- running episode 4 / 30
Run 6 -- running episode 4 / 30
Run 3 -- running episode 5 / 30
Run 2 -- running episode 5 / 30
Run 1 -- running episode 6 / 30
Run 4 -- running episode 5 / 30
Run 5 -- running episode 5 / 30
Run 7 -- running episode 5 / 30
Run 8 -- running episode 5 / 30
Run 6 -- running episode 5 / 30
Run 3 -- running episode 6 / 30
Run 2 -- running episode 6 / 30
Run 1 -- running episode 7 / 30
Run 4 -- running episode 6 / 30
Run 5 -- running episode 6 / 30
Run 7 -- running episode 6 / 30
Run 2 -- running episode 7 / 30
Run 3 -- running episode 7 / 30
Run 8 -- running episode 6 / 30
Run 6 -- running episode 6 / 30
Run 1 -- running episode 8 / 30
Run 4 -- running episode 7 / 30
Run 3 -- running episode 8 / 30
Run 1 --

/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transports_RL/Scripts/Simple_Cross/cross_RL/environment.py:326: UserWarning: API change now handles step as floating point seconds
  fixed_con.simulationStep(t)
/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transports_RL/Scripts/Simple_Cross/cross_RL/environment.py:326: UserWarning: API change now handles step as floating point seconds
  fixed_con.simulationStep(t)
/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transports_RL/Scripts/Simple_Cross/cross_RL/environment.py:326: UserWarning: API change now handles step as floating point seconds
  fixed_con.simulationStep(t)
/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transport

/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transports_RL/Scripts/Simple_Cross/cross_RL/environment.py:326: UserWarning: API change now handles step as floating point seconds
  fixed_con.simulationStep(t)
/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transports_RL/Scripts/Simple_Cross/cross_RL/environment.py:326: UserWarning: API change now handles step as floating point seconds
  fixed_con.simulationStep(t)
/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transports_RL/Scripts/Simple_Cross/cross_RL/environment.py:326: UserWarning: API change now handles step as floating point seconds
  fixed_con.simulationStep(t)
/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transport

/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transports_RL/Scripts/Simple_Cross/cross_RL/environment.py:326: UserWarning: API change now handles step as floating point seconds
  fixed_con.simulationStep(t)
/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transports_RL/Scripts/Simple_Cross/cross_RL/environment.py:326: UserWarning: API change now handles step as floating point seconds
  fixed_con.simulationStep(t)
/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transports_RL/Scripts/Simple_Cross/cross_RL/environment.py:326: UserWarning: API change now handles step as floating point seconds
  fixed_con.simulationStep(t)
/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transport

...

/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transports_RL/Scripts/Simple_Cross/cross_RL/environment.py:326: UserWarning: API change now handles step as floating point seconds
  fixed_con.simulationStep(t)
/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transports_RL/Scripts/Simple_Cross/cross_RL/environment.py:326: UserWarning: API change now handles step as floating point seconds
  fixed_con.simulationStep(t)
/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transports_RL/Scripts/Simple_Cross/cross_RL/environment.py:326: UserWarning: API change now handles step as floating point seconds
  fixed_con.simulationStep(t)
/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transport

/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transports_RL/Scripts/Simple_Cross/cross_RL/environment.py:326: UserWarning: API change now handles step as floating point seconds
  fixed_con.simulationStep(t)
/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transports_RL/Scripts/Simple_Cross/cross_RL/environment.py:326: UserWarning: API change now handles step as floating point seconds
  fixed_con.simulationStep(t)
/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transports_RL/Scripts/Simple_Cross/cross_RL/environment.py:326: UserWarning: API change now handles step as floating point seconds
  fixed_con.simulationStep(t)
/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transport

/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transports_RL/Scripts/Simple_Cross/cross_RL/environment.py:326: UserWarning: API change now handles step as floating point seconds
  fixed_con.simulationStep(t)
/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transports_RL/Scripts/Simple_Cross/cross_RL/environment.py:326: UserWarning: API change now handles step as floating point seconds
  fixed_con.simulationStep(t)
/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transports_RL/Scripts/Simple_Cross/cross_RL/environment.py:326: UserWarning: API change now handles step as floating point seconds
  fixed_con.simulationStep(t)
/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transport

/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transports_RL/Scripts/Simple_Cross/cross_RL/environment.py:326: UserWarning: API change now handles step as floating point seconds
  fixed_con.simulationStep(t)
/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transports_RL/Scripts/Simple_Cross/cross_RL/environment.py:326: UserWarning: API change now handles step as floating point seconds
  fixed_con.simulationStep(t)
/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transports_RL/Scripts/Simple_Cross/cross_RL/environment.py:326: UserWarning: API change now handles step as floating point seconds
  fixed_con.simulationStep(t)
/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transport

/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transports_RL/Scripts/Simple_Cross/cross_RL/environment.py:326: UserWarning: API change now handles step as floating point seconds
  fixed_con.simulationStep(t)
/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transports_RL/Scripts/Simple_Cross/cross_RL/environment.py:326: UserWarning: API change now handles step as floating point seconds
  fixed_con.simulationStep(t)
/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transports_RL/Scripts/Simple_Cross/cross_RL/environment.py:326: UserWarning: API change now handles step as floating point seconds
  fixed_con.simulationStep(t)
/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transport

#### Selecting best performer

In [20]:
# Get max evaluate results
res = tools.get_grid_search_results(log_path)
res.sort_values("RL_mean_delay")

,run_id,unfinished_runs,RL_mean_delay,fixed_mean_delay,reward,policy,eps,update_freq
4,7,0,47.608031,59.540695,balanced,epsGreedy,0.1,10000
0,3,0,47.749320,59.267360,balanced,epsGreedy,0.1,10000
6,6,0,48.747618,59.278495,balanced,linDecEpsGreedy,0.1,5000
3,4,0,53.049062,59.713607,balanced,linDecEpsGreedy,0.1,10000
5,8,0,53.138544,59.461362,balanced,linDecEpsGreedy,0.1,10000
7,5,0,58.737614,59.639216,balanced,epsGreedy,0.1,5000
2,2,0,59.659045,59.588876,balanced,linDecEpsGreedy,0.1,5000
1,1,0,59.696166,58.861902,balanced,epsGreedy,0.1,5000


#### Loading last checkpoint of selected performer 

In [35]:
run = 1 # selected run
sumo_RL = tools.load_last_model_checkpoint(log_path, run)

#### Evaluate using user interface

In [37]:
sumo_RL.evaluate(runs=1, use_gui=True)

Evaluate 1 -- running episode 1 / 1


FatalTraCIError: connection closed by SUMO